# Features extraction from 3D fluorescent foci

**Daryna Yakymenko** <br> <br> 
31-07-2025

*Created for Biological Data Science Summer School, <br> Uzhhorod, Ukraine, July 19 - August 2, 2025*

In [18]:
import numpy as np
from skimage.io import imread, imshow
from skimage.measure import regionprops_table
import pandas as pd
from scipy.stats import entropy
from skimage.measure import regionprops

In [13]:
import skimage
print(skimage.__version__)

0.25.2


In [29]:
# Wczytanie 3D surowego obrazu (np. TIFF z wieloma warstwami)
image = imread(r"./data/input/2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3.tif")  # (Zależnie od formatu, może być (Z,Y,X))

# Wczytanie label map (etykiety obiektów)
labels = imread(r"./data/labels/2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3_labels.tif")  # 3D tablica z etykietami, obiekty >0

In [40]:
def  objects_features(labels, image):
    props = regionprops_table(
        labels, 
        intensity_image=image,
        properties=('label', 'area', 'centroid', 'mean_intensity', 'max_intensity', 'min_intensity','intensity_std')
    )
    
    df_features = pd.DataFrame(props)

    #Entropy
    shape = np.array(labels.shape)
    center = shape / 2

    def entropy(intensities): #Shannon entropy for the histogram dictribution
        if len(intensities) == 0:
            return 0
        hist_range = (float(intensities.min()), float(intensities.max()))
        if hist_range[0] == hist_range[1]:
            hist_range = (hist_range[0], hist_range[0] + 1e-6)
        hist, _ = np.histogram(intensities, bins=256, range=hist_range, density=True)
        hist = hist[hist > 0]
        return -np.sum(hist * np.log2(hist))

    entropy_list = []
    sphericity_list = []

    # regionprops => surface_area
    props_full = regionprops(labels, intensity_image=image)

    for region in props_full:
        label = region.label
        mask = labels == label
        intensities = image[mask]
        entropy_list.append(entropy(intensities))

        #assess sphericity
        if hasattr(region, "surface_area"):
            V = region.area
            A = region.surface_area
            if A > 0:
                phi = (np.pi**(1/3))*(6*V)**(2/3)/A
            else:
                phi = np.nan
        else:
            phi = np.nan  # if skimage < 0.19

        sphericity_list.append(phi)

    df_features = pd.DataFrame(props)
    df_features.rename(columns={"area": "volume_vox"}, inplace=True)

    df_features['entropy'] = entropy_list
    df_features['sphericity'] = sphericity_list

    # total_intensity = mean_intensity * volume_vox
    df_features["total_intensity"] = df_features["mean_intensity"] * df_features["volume_vox"]

    background = np.median(image[labels == 0])
    df_features["background_corrected_intensity"] = (
    df_features["total_intensity"] - background * df_features["volume_vox"])

    voxel_volume_um3 = 0.06 * 0.06 * 0.12
    df_features["volume_um3"] = df_features["volume_vox"] * voxel_volume_um3
    return df_features

In [41]:
%time df = objects_features(labels, image)

CPU times: total: 3.92 s
Wall time: 3.95 s


In [38]:
df.to_csv("./data/object_features.csv", index=False)

In [39]:
df.head()

,label,volume_vox,centroid-0,centroid-1,centroid-2,mean_intensity,max_intensity,min_intensity,intensity_std,entropy,sphericity,total_intensity,background_corrected_intensity,volume_um3
0,1,892.0,54.252242,198.862108,198.477578,30.910314,200.0,0.0,35.957231,7.187043,NaN,27572.0,27572.0,0.385344
1,2,1204.0,45.354651,30.614618,127.901163,37.416113,160.0,0.0,33.796274,9.327431,NaN,45049.0,45049.0,0.520128
2,3,936.0,53.991453,135.886752,28.801282,33.924145,150.0,0.0,31.508573,9.496843,NaN,31753.0,31753.0,0.404352
3,4,1196.0,50.260870,140.798495,10.142977,21.843645,99.0,2.0,16.391218,10.934842,NaN,26125.0,26125.0,0.516672
4,5,620.0,55.711290,256.804839,316.366129,20.574194,90.0,0.0,20.136854,11.630000,NaN,12756.0,12756.0,0.267840
